In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
train_data = pd.read_csv('titles_and_lemmatized_titles_with_nltk_and_maru_train.csv', encoding='utf-8')
train_data.fillna('', inplace=True)
test_data = pd.read_csv('titles_and_lemmatized_titles_with_nltk_and_maru_test.csv', encoding='utf-8')
test_data.fillna('', inplace=True)

train_data.head()

,doc_id,pair_id,group_id,target,title,lemmatized_titles_with_nltk_and_maru
0,15731,1,1,0,ВАЗ 21213 | Замена подшипников ступицы | Нива,ваза 21213 замена подшипник ступица нива
1,14829,2,1,0,"Ваз 2107 оптом в Сочи. Сравнить цены, купить ...",ваза 2107 оптом сочи сравнить цена купить потр...
2,15764,3,1,0,Купить ступица Лада калина2. Трансмиссия - пер...,купить ступица лад калина2 трансмиссия переход...
3,17669,4,1,0,Классика 21010 - 21074,классика 21010 21074
4,14852,5,1,0,Ступица Нива — замена подшипника своими руками,ступица нива замена подшипник свой рука


In [61]:
last_group = train_data.group_id.iloc[-1]
max_words = 15
tf_feat = pd.DataFrame(columns=np.arange(max_words))
tf_weig = pd.DataFrame(columns=np.arange(max_words))
for group in range(1, last_group + 1):
    mask_gr = train_data.group_id == group
    documents = train_data.lemmatized_titles_with_nltk_and_maru[mask_gr]
    
    vect = TfidfVectorizer()
    tfidf_matrix = vect.fit_transform(documents)
    df = pd.DataFrame(tfidf_matrix.toarray(), columns = vect.get_feature_names())
    voc = np.array(df.mean(axis=0).sort_values(ascending=False)[:max_words].index)
    weights = np.array(df.mean(axis=0).sort_values(ascending=False)[:max_words].values)
    intersect_w = np.zeros((documents.shape[0], max_words))
    for i, title in enumerate(documents):
        words = title.strip().split()
        for j, word in enumerate(voc):
            intersect_w[i, j] = words.count(word)

    temp = intersect_w * weights
    tf_feat = tf_feat.append(pd.DataFrame(intersect_w))
    tf_weig = tf_weig.append(pd.DataFrame(temp))


print(tf_feat.shape)
print(tf_weig.iloc[:, :5].head())

(11690, 15)
          0         1         2         3         4
0  0.070806  0.067034  0.061894  0.061080  0.000000
1  0.070806  0.000000  0.000000  0.000000  0.041879
2  0.000000  0.000000  0.000000  0.122159  0.041879
3  0.000000  0.000000  0.000000  0.000000  0.000000
4  0.000000  0.067034  0.061894  0.061080  0.000000


In [56]:
tf_feat['sum_15'] = tf_weig.sum(axis=1)
tf_feat['sum_10'] = tf_weig.iloc[:, :10].sum(axis=1)
tf_feat['sum_5'] = tf_weig.iloc[:, :5].sum(axis=1)

In [57]:
tf_feat.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,sum_15,sum_10,sum_5
0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.297277,0.297277,0.260814
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.221368,0.147657,0.112685
2,0.0,0.0,0.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.235473,0.235473,0.164038
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
4,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.226470,0.226470,0.190008


In [58]:
with open(f'Parse_done/tfidf_train_titles_one_feat.csv', mode='w', encoding='utf-8') as f_csv:
    tf_feat.to_csv(f_csv, index=False)

In [59]:
tf_feat.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,sum_15,sum_10,sum_5
count,11690.000000,11690.000000,11690.000000,11690.000000,11690.000000,11690.000000,11690.000000,11690.000000,11690.000000,11690.000000,11690.000000,11690.000000,11690.000000,11690.000000,11690.000000,11690.000000,11690.000000,11690.000000
mean,0.472113,0.340633,0.257827,0.207271,0.165184,0.128400,0.113259,0.100257,0.090590,0.078015,0.071685,0.067579,0.066125,0.058340,0.054662,0.194154,0.185258,0.159984
std,0.750818,0.630559,0.473519,0.427554,0.455387,0.396351,0.373432,0.338392,0.371942,0.294363,0.269016,0.286374,0.357973,0.252325,0.237273,0.277068,0.270474,0.240063
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.073543,0.065339,0.052995
75%,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.273891,0.261256,0.222075
max,16.000000,15.000000,5.000000,3.000000,11.000000,15.000000,15.000000,9.000000,15.000000,6.000000,3.000000,10.000000,15.000000,4.000000,3.000000,3.516675,3.328973,2.214738


In [62]:
first_group = test_data.group_id.iloc[0]
last_group = test_data.group_id.iloc[-1]

max_words = 15
tf_feat_test = pd.DataFrame(columns=np.arange(max_words))
tf_weig = pd.DataFrame(columns=np.arange(max_words))

for group in range(first_group, last_group + 1):
    mask_gr = test_data.group_id == group
    documents = test_data.lemmatized_titles_with_nltk_and_maru[mask_gr]
    
    vect = TfidfVectorizer()
    tfidf_matrix = vect.fit_transform(documents)
    df = pd.DataFrame(tfidf_matrix.toarray(), columns = vect.get_feature_names())
    voc = np.array(df.mean(axis=0).sort_values(ascending=False)[:max_words].index)
    intersect_w = np.zeros((documents.shape[0], max_words))
    for i, title in enumerate(documents):
        words = title.strip().split()
        for j, word in enumerate(voc):
            intersect_w[i, j] = words.count(word)

    temp = intersect_w * weights
    tf_feat_test = tf_feat_test.append(pd.DataFrame(intersect_w))
    tf_weig = tf_weig.append(pd.DataFrame(temp))


print(tf_feat_test.shape)
print(tf_weig.iloc[:, :5].head())

(16627, 15)
          0         1         2        3    4
0  0.054945  0.050619  0.045869  0.00000  0.0
1  0.000000  0.000000  0.000000  0.04093  0.0
2  0.054945  0.050619  0.045869  0.00000  0.0
3  0.054945  0.050619  0.045869  0.00000  0.0
4  0.000000  0.000000  0.000000  0.04093  0.0


In [63]:
tf_feat_test['sum_15'] = tf_weig.sum(axis=1)
tf_feat_test['sum_10'] = tf_weig.iloc[:, :10].sum(axis=1)
tf_feat_test['sum_5'] = tf_weig.iloc[:, :5].sum(axis=1)

In [64]:
tf_feat_test.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,sum_15,sum_10,sum_5
0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.203793,0.187374,0.151433
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.040930,0.040930,0.040930
2,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.187374,0.187374,0.151433
3,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.187374,0.187374,0.151433
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.088781,0.088781,0.040930


In [65]:
with open(f'Parse_done/tfidf_test_titles_one_feat.csv', mode='w', encoding='utf-8') as f_csv:
    tf_feat_test.to_csv(f_csv, index=False)

In [66]:
tf_feat_test.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,sum_15,sum_10,sum_5
count,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000
mean,0.430625,0.326758,0.252962,0.203224,0.167860,0.142479,0.115655,0.096349,0.093583,0.081614,0.072773,0.066157,0.065857,0.063451,0.057978,0.086842,0.081772,0.066932
std,0.670238,0.664425,0.506308,0.464829,0.409976,0.435098,0.343049,0.338376,0.380758,0.330517,0.328664,0.261994,0.323959,0.300973,0.257231,0.103210,0.100329,0.085944
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.054945,0.054945,0.045869
75%,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.141505,0.135663,0.105564
max,15.000000,16.000000,14.000000,13.000000,7.000000,15.000000,7.000000,14.000000,15.000000,13.000000,13.000000,4.000000,15.000000,11.000000,6.000000,2.023604,2.023604,1.431730
